Using [BAAI/bge-reranker-v2-m3](https://huggingface.co/BAAI/bge-reranker-v2-m3) and langchain to test its performance.

In [1]:
import weaviate
import os

weaviate_client = weaviate.Client("http://localhost:8080",
    additional_headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_APIKEY"),
    }
)

c:\anaconda\envs\dspy\Lib\site-packages\weaviate\warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.2. The latest version is 4.5.5.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core.response.notebook_utils import display_response
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

In [9]:
import requests

url = 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt'
r = requests.get(url)

file_path = 'example/paul_graham/paul_graham_essay.txt'
os.makedirs(os.path.dirname(file_path), exist_ok=True)

with open(file_path, 'wb') as f:
    f.write(r.content)

In [13]:
documents = SimpleDirectoryReader("example/paul_graham/").load_data()

In [18]:
from llama_index.core import StorageContext
from llama_index.llms.openai import OpenAI

## config llm like this
llm = OpenAI(model="gpt-4")

vector_store = WeaviateVectorStore(weaviate_client=weaviate_client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, index_name="test_prefix"
)

query_engine = index.as_query_engine(llm=llm, similarity_top_k=5) ## from this part check llm.
response = query_engine.query("What did the author do growing up?")

In [19]:
print(response)

The author spent his time outside of school writing and programming. He wrote short stories and began programming on an IBM 1401 in 9th grade. He used an early version of Fortran to write programs on punch cards. Later, he started programming on a microcomputer, a TRS-80, which his father bought for him. He wrote simple games, a program to predict the trajectory of his model rockets, and a word processor that his father used to write a book. Despite his interest in programming, he initially planned to study philosophy in college.


In [ ]:
print(response.get_formatted_sources(length=200))

With Reranker

In [22]:
rerank = FlagEmbeddingReranker(model="BAAI/bge-reranker-v2-m3", top_n=5)

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

c:\anaconda\envs\dspy\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\송준호\.cache\huggingface\hub\models--BAAI--bge-reranker-v2-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

In [ ]:
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5, node_postprocessors=[rerank]) ## from this part check llm.
response = query_engine.query("What did the author do growing up?")

In [ ]:
print(response)

In [ ]:
print(response.get_formatted_sources(length=200))